In [ ]:
import os
os.chdir('/content/drive/MyDrive/DataDrive2030 Early Learning Predictors Challenge')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
!pip install catboost -q

In [ ]:
!pip install shap -q

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import shap

In [ ]:
Train = pd.read_csv('Train_DD.csv')
Test = pd.read_csv('Test_DD.csv')
Sub = pd.read_csv('SampleSubmission.csv')
VarD = pd.read_csv('VariableDescription.csv')

In [ ]:
Train.head()

In [ ]:
Test.head()

In [ ]:
pd.set_option('display.max_colwidth', 100)
VarD

In [ ]:
vard = VarD.set_index('Variable Name').T

In [ ]:
def plot_missing_cols(df, threshold=0.8, plot=True):
  missing_per = df.isnull().sum()/len(df)
  missing_per_thresh = missing_per[missing_per>threshold]
  if plot:
    plt.figure(figsize=(20,10))
    plt.barh(missing_per_thresh.index, missing_per_thresh.values * 100)
    plt.xlabel('Percentage of missing values')
    plt.title(f'Missing percentage above {threshold*100}% theshold')
    plt.show()
  return missing_per_thresh.index



#Feature Visualization

In [ ]:
def scatterplot(x,y,hue=None,length=5,width=10):
  fig, ax = plt.subplots(figsize=(width,length))
  sns.scatterplot(x=x,y=y,hue=hue,ax=ax)
  ax.set_xlabel(x.name)
  ax.set_ylabel(y.name)
  ax.set_title(f'{x.name} vs {y.name}')
  plt.show()

In [ ]:
def boxplot(df):
  fig, ax = plt.subplots()
  ax.boxplot(df)
  ax.set_xticklabels(list(df.columns))
  ax.set_ylabel('Value')
  ax.set_title(f'Boxplot of {" vs ".join(list(df.columns))}')
  plt.show()

In [ ]:
def heatmap(df, length=10, width=5, cmap='rocket'):
  heatmap_df = df.corr()
  fig, ax = plt.subplots(figsize=(length, width))
  sns.heatmap(data=heatmap_df, annot=True, cmap=cmap, ax=ax)
  ax.set_title(f'Heatmap of {" and ".join(list(df.columns))}')
  plt.show()

In [ ]:
def violinplot(x, y,hue,split=False,scale=None,length=10, width=5):
  fig, ax = plt.subplots(figsize=(length,width))
  sns.violinplot(x=x,y=y,hue=hue,split=split,scale=scale,ax=ax)
  ax.set_title(f'ViolinPlot of {x.name} and {y.name}')
  ax.set_xlabel(x.name)
  ax.set_ylabel(y.name)
  plt.legend(title=hue.name)
  plt.show()

In [ ]:
def barchart(x,y,length=10,width=5):
  fig, ax = plt.subplots(figsize=(length,width))
  sns.barplot(x=x,y=y,ax=ax)
  ax.set_title(f'Bar Chart of {x.name} and {y.name}')
  ax.set_xlabel(x.name)
  ax.set_ylabel(y.name)
  plt.show()

In [ ]:
def stack_barchart(x,y,hue,length=10,width=5):
  fig, ax = plt.subplots(figsize=(length,width))
  sns.barplot(x=x,y=y,hue=hue,ax=ax)
  ax.set_title(f'Bar Chart of {x.name} and {y.name}')
  ax.set_xlabel(x.name)
  ax.set_ylabel(y.name)
  plt.legend(title=hue.name)
  plt.show()

In [ ]:
Train['Train'] = 'Train'
Test['Train'] = 'Test'
Data = pd.concat([Train, Test])
data = Data.drop(['child_id'],axis=1)

In [ ]:
def targetmap(age, target):
  if (age >= 49) and (age < 60):
    if (target >= 0) and (target < 36.02):
      return "At Risk"
    elif (target >= 36.02) and (target < 46.32):
      return "Falling Behind"
    elif (target >= 46.32) and (target < 101):
      return "Achieving the Standard"

  elif (age >= 60) and (age < 70):
    if (target >= 0) and (target < 43.24):
      return "At Risk"
    elif (target >= 43.24) and (target < 54.38):
      return "Falling Behind"
    elif (target >= 54.38) and (target < 101):
      return "Achieving the Standard"

In [ ]:
Train['target_cat'] = Train[['child_age','target']].apply(lambda row:targetmap(row[0],row[1]),axis=1)

In [ ]:
#Checking out the Children Columns
Train.filter(like='child').head(3)

##child_columns

In [ ]:
vard[Train.filter(like='child').columns]

###child grant

In [ ]:
Train['child_grant'].isnull().sum()/len(Train)

In [ ]:
barchart(Train['child_grant'],Train['target'])

In [ ]:
stack_barchart(Train['child_grant'],Train['target'],hue=Train['target_cat'])
#Children whose primary caretaker refused the Child Support Grant have no child At Risk
#Children whose primary caretaker refused the Child Support Grant have more children achieving the standard

In [ ]:
Train['child_grant'].value_counts()

In [ ]:
Train[Train['child_grant']=='Yes']['target_cat'].value_counts()/len(Train[Train['child_grant']=='Yes'])

In [ ]:
Train[Train['child_grant']=='No']['target_cat'].value_counts()/len(Train[Train['child_grant']=='No'])

In [ ]:
Train[Train['child_grant']=="Don't know"]['target_cat'].value_counts()/len(Train[Train['child_grant']=="Don't know"])

In [ ]:
""" Child Support Grant: 'No' - Majority of those who responded 'No' for the Child Support Grant are achieving the standard,
which may indicate that they do not rely on the grant for financial assistance and are meeting the standard of living without it.
A smaller percentage are at risk or falling behind, which could suggest that they may not require the grant due to their current financial situation.

Child Support Grant: 'Yes' - Majority of those who responded 'Yes' for the Child Support Grant are achieving the standard,
which may indicate that they rely on the grant for financial assistance and are able to meet the standard of living with its support.
A significant percentage are falling behind, which could suggest that the grant may not be sufficient to lift them out of poverty entirely.

Child Support Grant: 'Don't Know' - Those who responded 'Don't Know' for the Child Support Grant are relatively smaller in number compared to 'No' and 'Yes'.
Majority of them are falling behind, which could suggest that they may not have adequate knowledge or information about the grant,
its eligibility criteria, or how to apply for it, resulting in uncertainty about their eligibility status."""
pass

###child age

In [ ]:
Train['child_age'].isnull().sum()/len(Train)

In [ ]:
Train[['child_age','target']]

In [ ]:
scatterplot(Train['child_age'],Train['target'])
boxplot(Train[['child_age','target']])

#Child age doesn't seem to be correlated to the target, maybe not alone

In [ ]:
violinplot(x=Train['child_grant'],y=Train['child_age'],hue=Train['target_cat'],scale='count')

In [ ]:
Train['age_bin'] = pd.qcut(Train['child_age'],5,labels=False)

In [ ]:
barchart(Train['age_bin'],Train['target'])

In [ ]:
stack_barchart(Train['age_bin'],Train['target'],hue=Train['target_cat'])

In [ ]:
grouped = Train.groupby(['age_bin','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['age_bin']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
"""These insights provide information on the distribution of target categories (i.e., "Achieving the Standard", "Falling Behind", "At Risk") among different age groups (i.e., age bins).
It suggests that the proportion of children in each target category varies across different age groups, with the proportion of children
categorized as "Achieving the Standard" generally increasing with age, while the proportion of children categorized as "At Risk"
generally decreasing with age."""
pass

###child enrollment

In [ ]:
Train.filter(like='enrolment')

In [ ]:
Train.filter(like='enrolment').isnull().sum()/len(Train)

In [ ]:
Train.filter(like='date')
#It seem child_enrolment_date + child_months_enrolment gives the day the child's ELOM was taken (child_date)

In [ ]:
Train.filter(like='date').isnull().sum()/len(Train)

In [ ]:
vard[Train.filter(like='date').columns]
#What do the dates mean? and how are they related?

In [ ]:
heatmap(Train[['child_months_enrolment','target']])

In [ ]:
Train.corr()['child_months_enrolment'].sort_values(ascending=False)[:10]

In [ ]:
Train[['count_practitioners_age_1','count_register_year_2019', 'count_register_year_2018', 'id_enumerator',
      'count_register_year_2020', 'pri_expense_food','count_practitioners_age_0', 'count_practitioners_age_3','obs_classrooms']].isnull().sum()/len(Train)

In [ ]:
#We are missing about 69% of the Child Enrolment Dates? Should we drop the columns?
#There doesn't seem to be an accurate way of replacing the missing dates
#Dropping all columns with missing values up to the 80th percentile
#There doesn't seem to be a positive correlation with months enrolment with other features, CHECK BACK LATER

###child_years_in_programme

In [ ]:
Train['child_years_in_programme'].isnull().sum()/len(Train)

In [ ]:
Train['child_years_in_programme'].value_counts()

In [ ]:
barchart(Train['child_years_in_programme'], Train['target'],10,5)

In [ ]:
stack_barchart(Train['child_years_in_programme'],Train['target'],hue=Train['target_cat'],length=10,width=5)
#The third year seems to have the most uneven distribution, it has the smallest distribution of the classes of child_grant
#The second years seems to have the most even distribution

In [ ]:
grouped = Train.groupby(['child_years_in_programme','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['child_years_in_programme']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
stack_barchart(Train['child_years_in_programme'],Train['target'],hue=Train['age_bin'],length=10,width=5)
#The children seem to be evenly distributed by age groups in the program years

In [ ]:
grouped = Train.groupby(['child_years_in_programme','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = Train.groupby(['child_years_in_programme']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
"""Age may impact program participation: The distribution of age groups may impact the number of years children spend in the programme.
For example, children in age bin 0 (likely younger children) have higher proportions in the first and third years of the programme,
while children in age bin 1 (slightly older children) have a higher proportion in the second year of the programme.
This suggests that age may impact the duration of participation in the programme, with younger children potentially spending more time
in the programme compared to older children."""
pass

###child_height

In [ ]:
Train['child_height'].isnull().sum()/len(Train)

In [ ]:
Train['child_height']

In [ ]:
sns.scatterplot(x=Train['child_height'],y=Train['target'],hue=Train['target_cat'])

In [ ]:
Train.corr()['child_height'].sort_values(ascending=False)[:15]

In [ ]:
vard['child_zha']

In [ ]:
scatterplot(x=Train['child_zha'],y=Train['target'])

In [ ]:
Train.corr()['child_zha'].sort_values(ascending=False)[:15]

###child_observe

In [ ]:
Train['child_observe_attentive'].isnull().sum()/len(Train)

In [ ]:
Train['child_observe_attentive'].value_counts()

In [ ]:
barchart(x=Train['child_observe_attentive'],y=Train['target'])

In [ ]:
violinplot(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['target_cat'],scale='count',length=15)

In [ ]:
grouped = Train.groupby(['child_observe_attentive','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['child_observe_attentive']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
stack_barchart(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['age_bin'])

In [ ]:
violinplot(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['age_bin'],scale='count',length=20)

In [ ]:
grouped = Train.groupby(['child_observe_attentive','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = Train.groupby(['child_observe_attentive']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
"""These insights suggest that higher levels of attentiveness ("Often" and "Almost always") are associated with a higher proportion of
children achieving the standard, while lower levels of attentiveness ("Sometimes" and "Almost never") are associated with a higher
proportion of children being at risk or falling behind. This implies that attentiveness may be an important factor influencing children's
academic performance or progress.
These insights suggest that there may be some variation in the distribution of age groups across different levels of child observance of attentiveness.
Further analysis or statistical tests may be needed to determine if these differences are statistically significant and to understand any potential relationships
between child attentiveness, age, and other factors.
Based on the given insights, the age group that appears to be most attentive is age group 4, as it has the highest proportion of children
who are observed to be attentive 'Almost always'. On the other hand, the age group that appears to be least attentive is age group 3,
as it has the lowest proportion of children who are observed to be attentive 'Sometimes', 'Often', and 'Almost always'.
However, it's important to note that these conclusions are based on the data provided and may not necessarily indicate a
definitive pattern or relationship. Further analysis and statistical tests may be needed to draw more conclusive insights."""
pass

In [ ]:
stack_barchart(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['child_years_in_programme'])

In [ ]:
violinplot(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['child_years_in_programme'],scale='count',length=20)

In [ ]:
grouped = Train.groupby(['child_observe_attentive','child_years_in_programme'])[['child_years_in_programme']].count().rename(columns={'child_years_in_programme':'count'})
total_counts = Train.groupby(['child_observe_attentive']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
att = Train[Train['child_observe_attentive']=='Sometimes']
att1 = Train[Train['child_observe_attentive']=='Often']
att2 = Train[Train['child_observe_attentive']=='Almost always']
att3 = Train[Train['child_observe_attentive']=='Almost never']


In [ ]:
violinplot(x=att['child_years_in_programme'],y=att['target'],hue=att['age_bin'],scale='count',length=20)

In [ ]:
violinplot(x=att1['child_years_in_programme'],y=att1['target'],hue=att1['age_bin'],scale='count',length=20)

In [ ]:
violinplot(x=att2['child_years_in_programme'],y=att2['target'],hue=att2['age_bin'],scale='count',length=20)

In [ ]:
violinplot(x=att3['child_years_in_programme'],y=att3['target'],hue=att3['age_bin'],scale='count',length=20)

In [ ]:
grouped = att.groupby(['child_years_in_programme','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = att.groupby(['child_years_in_programme']).size()
grouped['Sometimes'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
sm = grouped.drop('count',axis=1)

In [ ]:
grouped = att1.groupby(['child_years_in_programme','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = att1.groupby(['child_years_in_programme']).size()
grouped['Often'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
oft = grouped.drop('count',axis=1)

In [ ]:
grouped = att2.groupby(['child_years_in_programme','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = att2.groupby(['child_years_in_programme']).size()
grouped['Almost always'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
alma = grouped.drop('count',axis=1)

In [ ]:
grouped = att3.groupby(['child_years_in_programme','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = att3.groupby(['child_years_in_programme']).size()
grouped['Almost never'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
almn = grouped.drop('count',axis=1)

In [ ]:
pd.concat([sm,oft,alma,almn],axis=1)

In [ ]:
"""Older Kids tend to pay more attention than other age groups, but Older Kids are mostly only present in the First Year of the Programme
and the category "Don't Know". The age group 1 pays more attentions that the other groups in the Second Year and Third Year of the Programme,
While Group 0 which are the youngest set of children most often don't pay attention, compared to others in the Third Year of the Programme."""
pass

In [ ]:
stack_barchart(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['child_grant'])

In [ ]:
violinplot(x=Train['child_observe_attentive'],y=Train['target'],hue=Train['child_grant'],scale='count',length=20)

In [ ]:
grouped = Train.groupby(['child_observe_attentive','child_grant'])[['child_grant']].count().rename(columns={'child_grant':'count'})
total_counts = Train.groupby(['child_observe_attentive']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop('count',axis=1,inplace=True)
grouped

In [ ]:
Train['child_observe_concentrated'].isnull().sum()/len(Train)

In [ ]:
Train['child_observe_concentrated'].value_counts()

In [ ]:
Train.filter(like='observe').columns

In [ ]:
vard[Train.filter(like='observe').columns]

In [ ]:
barchart(Train['child_observe_concentrated'],Train['target'])

In [ ]:
stack_barchart(Train['child_observe_concentrated'],Train['target'],hue=Train['target_cat'])

In [ ]:
violinplot(Train['child_observe_concentrated'],Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
"""We can Decipher from the ViolinPlot, that Children who "Almost always" stay concentrated on task during the activities have the highest
possible performance Achieving the Standard and are hardly At Risk, followed by Children who "Often" stay concentrated on task during the activities
those who "Almost never" stay concentrated have a higher chance of beeen At Risk and have a lower chance of even Acheiving the Standards similar
to those who "Sometimes" stay concentrated"""
pass

In [ ]:
violinplot(Train['child_observe_attentive'],Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
violinplot(Train['child_observe_diligent'],Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
violinplot(Train['child_observe_interested'],Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
grouped = Train.groupby(['child_observe_attentive', 'target_cat'])[['target_cat']].count().rename(columns={'target_cat': 'count'})
total_counts = Train.groupby('child_observe_attentive').size()
grouped['attentive'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
att= grouped.drop(['count'],axis=1)
# grouped = grouped.reset_index()

In [ ]:
grouped = Train.groupby(['child_observe_interested', 'target_cat'])[['target_cat']].count().rename(columns={'target_cat': 'count'})
total_counts = Train.groupby('child_observe_interested').size()
grouped['interested'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
interest = grouped.drop(['count'],axis=1)
# grouped = grouped.reset_index()

In [ ]:
grouped = Train.groupby(['child_observe_concentrated', 'target_cat'])[['target_cat']].count().rename(columns={'target_cat': 'count'})
total_counts = Train.groupby('child_observe_concentrated').size()
grouped['concentrated'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
conc = grouped.drop(['count'],axis=1)
# grouped = grouped.reset_index()

In [ ]:
grouped = Train.groupby(['child_observe_diligent', 'target_cat'])[['target_cat']].count().rename(columns={'target_cat': 'count'})
total_counts = Train.groupby('child_observe_diligent').size()
grouped['diligent'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
dilig = grouped.drop(['count'],axis=1)
# grouped = grouped.reset_index()

In [ ]:
pd.concat([att,conc,dilig,interest],axis=1)

In [ ]:
#child_observe_diligent > child_observe_attentive > child_observe_interested > child_observe_concentrated

In [ ]:
violinplot(x=Train['child_observe_concentrated'],y=Train['child_age'],hue=Train['age_bin'],scale='count')

In [ ]:
violinplot(x=Train['child_observe_attentive'],y=Train['child_age'],hue=Train['age_bin'],scale='count')

In [ ]:
violinplot(x=Train['child_observe_diligent'],y=Train['child_age'],hue=Train['age_bin'],scale='count')

In [ ]:
violinplot(x=Train['child_observe_interested'],y=Train['child_age'],hue=Train['age_bin'],scale='count')

In [ ]:
"""In all different categories of observation, it seems like age_group 4 are more dominant in the Almost Always category, and
age_group 0 are more dominant in the Almost never category, this shows that older aged children tend to show more interest, diligence,
concentration and attention, while younger aged children seem to be more carefree"""
pass

In [ ]:
sns.stripplot(x=Train['child_observe_total'],y=Train['target'],hue=Train['target_cat'],jitter=True)

In [ ]:
sns.stripplot(x=Train['child_observe_total'],y=Train['child_age'],hue=Train['target_cat'],jitter=True)

In [ ]:
sns.stripplot(x=Train['child_observe_total'],y=Train['child_height'],hue=Train['target_cat'],jitter=True)

###child_gender

In [ ]:
Train['child_gender'].isnull().sum()/len(Train)

In [ ]:
Train['child_gender'].value_counts()

In [ ]:
barchart(x=Train['child_gender'],y=Train['target'])

In [ ]:
violinplot(x=Train['child_gender'],y=Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
grouped = Train.groupby(['child_gender','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['child_gender']).size()
grouped['normalized_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped

In [ ]:
"""It seems like Female Kids are more likely to Achieve the standard than Male Kids, but Male kids have a higher chance to be At Risk and Falling Behind"""
pass

In [ ]:
violinplot(x=Train['child_observe_diligent'],y=Train['target'],hue=Train['child_gender'],split=True,scale='count')

In [ ]:
"""The Female gender has a more spread out distribution than the male counterpart, for the attention levels Almost always and Often, and have
a smaller distribution than their male counterparts for the attention levels Sometimes and Often"""
pass

In [ ]:
violinplot(x=Train['child_observe_diligent'],y=Train['child_age'],hue=Train['child_gender'],split=True,scale='count')

In [ ]:
""""The Female gender has a more spread out distribution than the male counterpart, for the attention levels Almost always and Often, and have
a smaller distribution than their male counterparts for the attention levels Sometimes and Often"""
pass

###child_dob

In [ ]:
Train['child_dob'].isnull().sum()/len(Train)

In [ ]:
Train['Year_dob'] = Train['child_dob'].str.split('-',expand=True)[0].astype(float)
Train['Month_dob'] = Train['child_dob'].str.split('-',expand=True)[1].astype(float)
Train['Day_dob'] = Train['child_dob'].str.split('-',expand=True)[2].astype(float)


In [ ]:
Train['Year_dob'].value_counts()

In [ ]:
Train['Month_dob'].value_counts()

In [ ]:
Train['Day_dob'].value_counts()

In [ ]:
heatmap(Train[['Year_dob','Month_dob','Day_dob','child_age','age_bin']])

In [ ]:
barchart(Train['Year_dob'],Train['target'])

In [ ]:
violinplot(Train['Year_dob'],Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
grouped = Train.groupby(['Year_dob','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['Year_dob']).size()
grouped['normalized_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped

In [ ]:
grouped = grouped.reset_index()
sns.catplot(data=grouped,x='Year_dob',y='normalized_count',hue='target_cat',kind='point')

In [ ]:
"""There seems to be an upward and downward trend among the Years, the target performance improved from the year 2015 - 2016 the became worse in the year 2017
the become better for those born in the year 2018, why is there a downwards trend between children born in 2016-2017, and why is there an upwards trend
between those born in 2017-2018, same happened in 2015-2016"""
pass

In [ ]:
grouped = Train.groupby(['Year_dob','Month_dob','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['Year_dob','Month_dob']).size()
xy=pd.DataFrame(total_counts,columns=['size'])
xyz = pd.merge(grouped.reset_index(), xy.reset_index(), left_on=['Year_dob', 'Month_dob'], right_on=['Year_dob', 'Month_dob'])
xyz['count'] = xyz['count']/xyz['size']
xyz

In [ ]:
sns.catplot(data=xyz,x='Month_dob',y='count',col='Year_dob',hue='target_cat',kind='point',height=6)

In [ ]:
#There doesn't seem to be a pattern, just a random distribution

###child_zha

In [ ]:
Train['child_zha'].isnull().sum()/len(Train)

In [ ]:
sns.scatterplot(x=Train['child_zha'],y=Train['target'],hue=Train['target_cat'])

In [ ]:
barchart(x=Train['child_gender'],y=Train['child_zha'])

In [ ]:
barchart(x=Train['age_bin'],y=Train['child_zha'])

###child_stunted

In [ ]:
Train['child_stunted'].isnull().sum()/len(Train)

In [ ]:
Train['child_stunted'].value_counts()

In [ ]:
barchart(x=Train['child_stunted'],y=Train['target'])

In [ ]:
violinplot(x=Train['child_stunted'],y=Train['target'],hue=Train['target_cat'],scale='count')

In [ ]:
grouped = Train.groupby(['child_stunted','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_count = Train.groupby(['child_stunted']).size()
grouped['norm_count'] = grouped['count']/total_count[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped

In [ ]:
barchart(x=Train['child_stunted'],y=Train['child_height'])

In [ ]:
barchart(x=Train['child_stunted'],y=Train['child_zha'])

In [ ]:
violinplot(x=Train['child_stunted'],y=Train['child_age'],hue=Train['age_bin'],scale='count')

In [ ]:
grouped = Train.groupby(['child_stunted','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = Train.groupby(['child_stunted']).size()
grouped['norm_count'] = grouped['count']/total_count[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped

####child_attends

In [ ]:
Train['child_attends'].isnull().sum()/len(Train)

####child_attendance

In [ ]:
Train['child_attendance'].isnull().sum()/len(Train)

####child_languages

In [ ]:
Train['child_languages'].isnull().sum()/len(Train)

###child_age_group

In [ ]:
Train['child_age_group'].isnull().sum()/len(Train)

In [ ]:
Train['child_age_group'].value_counts()

In [ ]:
Train.groupby(['child_age_group','age_bin'])[['age_bin']].count()

In [ ]:
grouped = Train.groupby(['child_age_group','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['child_age_group']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped


####pri_children_4_6_years

In [ ]:
Train['pri_children_4_6_years'].isnull().sum()/len(Train)

####obs_toilets_children


In [ ]:
Train['obs_toilets_children'].isnull().sum()/len(Train)

In [ ]:
Train['obs_toilets_children'].value_counts()

###count_

In [ ]:
Train['count_children_present'].isnull().sum()/len(Train)
Test['count_children_present'].isnull().sum()/len(Test)

In [ ]:
count_cols = Train.filter(like='count')

In [ ]:
missing_cols=plot_missing_cols(count_cols,plot=True)

In [ ]:
count_cols = count_cols.drop(missing_cols,axis=1)

#####count_staff

In [ ]:
vard[count_cols.filter(like='staff').columns]
#Count_staff_all and Count_staff_gender are the same thing, Count_staff_salary is the total number of paid staffs
#count_staff_contract and	count_staff_time are the same, count_staff_time_full

In [ ]:
Train['count_staff_skills'] = Train['count_staff_qual_skills']*0.2+Train['count_staff_qual_nqf4_5']*0.2+Train['count_staff_qual_nqf6_9']*0.6

In [ ]:
Train.groupby(['child_grant'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
#Staff Qualifications: The count of staff with skills, NQF Level 4-5 qualification, and NQF Level 6-9 qualification has an impact on the
#child_grant category. The 'count_staff_skills' metric, which is a weighted combination of these qualifications, is highest for the
#'No' category of child_grant, indicating that programs with staff having higher qualifications tend to have fewer children receiving the
#Child Support Grant.

#Staff Salary: The average count of staff salary paid is highest for the 'No' category of child_grant, indicating that programs where
#children do not receive the Child Support Grant tend to have higher staff salaries.

In [ ]:
Train.groupby(['age_bin'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
#Overall, the data suggests that as the age group category increases, the staff qualifications, staff salary, proportion of female staff
#members, and target outcome tend to increase as well. This could indicate that programs targeting older children may have different
#staffing and operational requirements compared to programs targeting younger children.

In [ ]:
Train.groupby(['Year_dob'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()

In [ ]:
Train.groupby(['child_years_in_programme'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
#Staff Qualifications: The average count of staff skills ('count_staff_skills') tends to increase slightly from the 1st year to the 3rd
#year in the program, with the highest average observed in the 3rd year. This suggests that staff qualifications may improve over time as
#they gain more experience in the program.

#Staff Salary: The average count of staff salary paid ('count_staff_salary_paid') also tends to increase from the 1st year to the
#3rd year in the program, indicating that staff may receive higher salaries as they progress in their tenure.

#Gender Distribution: The average count of female staff members ('count_staff_gender_female') does not show a significant trend
#across different years of enrollment in the program, as the values are relatively similar.

#Target Outcome: The average target outcome, which is not defined in the given data, shows a slight fluctuation across different
#years of enrollment in the program, with the highest average observed in the "Do Not Know" category.

In [ ]:
aa = Train.groupby(['child_observe_attentive'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
ab = Train.groupby(['child_observe_interested'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
ac = Train.groupby(['child_observe_concentrated'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()
ad = Train.groupby(['child_observe_diligent'])[['count_staff_skills','count_staff_salary_paid','count_staff_gender_female','target']].mean()

ab = aa.join(ab,lsuffix='_aa',rsuffix='_ab')
ad = ac.join(ad,lsuffix='_ac',rsuffix='_ad')
at = ab.join(ad)
at['average_staff_skills'] = np.mean(at.filter(like='skills'),axis=1).values
at['average_staff_salary_paid'] = np.mean(at.filter(like='salary'),axis=1).values
at['average_staff_gender_female'] = np.mean(at.filter(like='gender'),axis=1).values
at['average_target'] = np.mean(at.filter(like='target'),axis=1).values

at = at.filter(like='average')
at

#Overall, the data suggests that child observation levels may have an impact on staff qualifications, staff salary, and target outcome.

In [ ]:
sns.catplot(data=Train,x='count_staff_salary_paid',y='target',kind='boxen',height=10)

In [ ]:
sns.catplot(data=Train,x='count_staff_gender_female',y='target',kind='boxen',height=10)
#A lot of Outliers present in both count_staff_gender_female and target variable, is there a reason why?

#####count_register

In [ ]:
vard[count_cols.filter(like='register').columns]
#count_register_all and count_register_gender are the same

In [ ]:
count_cols.filter(like='race')

In [ ]:
count_cols.filter(like='race').describe()

In [ ]:
Train.groupby(['Year_dob'])[['count_register_race_african','count_register_race_coloured',
                             'count_register_race','target']].mean()

In [ ]:
Train.groupby(['child_grant'])[['count_register_race_african','count_register_race_coloured',
                             'count_register_race','target']].mean()
#Child Grant and Count Register: There are differences in the average counts of children registered in the ECD Programme
#("count_register_race") based on whether the child is receiving a grant or not. Specifically, children who are reported as
#"Don't know" or "Refuse" regarding their grant status tend to have higher average counts of African/Black children
#("count_register_race_african") compared to children who are not receiving a grant ("No") or are receiving a grant ("Yes").
#This suggests that there may be some association between grant status and the counts of African/Black children in the ECD Programme.

In [ ]:
"""Count of Registered Children: The average counts of African/Black children ("count_register_race_african"), Coloured children
("count_register_race_coloured"), and the overall count of registered children ("count_register_race") show some variability across
different age bins. Specifically, the counts tend to decrease from age bin 0 to age bin 4. This suggests that there may be some
 differences in the enrollment or registration of children from different racial/ethnic backgrounds based on their age, with
 lower counts observed in higher age bins."""

Train.groupby(['age_bin'])[['count_register_race_african','count_register_race_coloured',
                             'count_register_race','target']].mean()


In [ ]:
Train.groupby(['child_years_in_programme'])[['count_register_race_african','count_register_race_coloured',
                             'count_register_race','target']].mean()
#The number of African children increases over the years, while the number of coloured children decreases over the years

In [ ]:
aa = Train.groupby(['child_observe_attentive'])[['count_register_race_african','count_register_race_coloured','count_register_race','target']].mean()
ab = Train.groupby(['child_observe_interested'])[['count_register_race_african','count_register_race_coloured','count_register_race','target']].mean()
ac = Train.groupby(['child_observe_concentrated'])[['count_register_race_african','count_register_race_coloured','count_register_race','target']].mean()
ad = Train.groupby(['child_observe_diligent'])[['count_register_race_african','count_register_race_coloured','count_register_race','target']].mean()

ab = aa.join(ab,lsuffix='_aa',rsuffix='_ab')
ad = ac.join(ad,lsuffix='_ac',rsuffix='_ad')
at = ab.join(ad)
at['average_register_race_african'] = np.mean(at.filter(like='african'),axis=1).values
at['average_register_race_coloured'] = np.mean(at.filter(like='coloured'),axis=1).values
at['average_target'] = np.mean(at.filter(like='target'),axis=1).values

at = at.filter(like='average')
at

In [ ]:
Train['Black-Coloured-Ratio'] = Train['count_register_race'] / Train['count_register_race_african']

In [ ]:
Train['Black-Coloured-Ratio'] = Train['Black-Coloured-Ratio'].replace(np.inf, 0)

In [ ]:
count_cols.filter(like='register_year').describe()

In [ ]:
Train['count_register_year_total'] = count_cols.filter(like='register_year').drop(['count_register_year_grader','count_register_year_school'],axis=1).sum(axis=1)

In [ ]:
Train.groupby(['Year_dob'])[['count_register_all','count_register_year_grader',
                             'count_register_year_school','target']].mean()

In [ ]:
Train.groupby(['child_grant'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()

In [ ]:
Train.groupby(['age_bin'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()

In [ ]:
Train.groupby(['child_years_in_programme'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()
#Participation in the ECD Programme: Children who have been in the programme for longer durations (2nd and 3rd year) tend to have
#slightly higher average counts of children registered in the ECD Programme ("count_register_all") compared to children in their
#1st year in the programme. This suggests that there may be an increasing trend in participation in the ECD Programme over time.

#Grade R Participation: Children who have been in the programme for longer durations (2nd and 3rd year) tend to have slightly
#higher average counts of children registered in Grade R ("count_register_year_grader") compared to children in their 1st year in
#the programme. This indicates that children who have been in the programme for multiple years are more likely to progress to Grade R,
#which is a higher level of early childhood education.

#School Age Participation: Children who have been in the programme for longer durations (2nd and 3rd year) tend to have slightly higher
#average counts of children registered in school age ("count_register_year_school") compared to children in their 1st year in the
#programme. This suggests that children who have been in the programme for multiple years are more likely to reach the school age and
#continue their education.

In [ ]:
aa = Train.groupby(['child_observe_attentive'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()
ab = Train.groupby(['child_observe_interested'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()
ac = Train.groupby(['child_observe_concentrated'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()
ad = Train.groupby(['child_observe_diligent'])[['count_register_all','count_register_year_grader','count_register_year_school','target']].mean()

ab = aa.join(ab,lsuffix='_aa',rsuffix='_ab')
ad = ac.join(ad,lsuffix='_ac',rsuffix='_ad')
at = ab.join(ad)
at['average_register_all'] = np.mean(at.filter(like='all'),axis=1).values
at['average_year_grader'] = np.mean(at.filter(like='grader'),axis=1).values
at['average_year_school'] = np.mean(at.filter(like='school'),axis=1).values
at['average_target'] = np.mean(at.filter(like='target'),axis=1).values

at = at.filter(like='average')
at

In [ ]:
heatmap(Train[['count_register_all','count_register_gender_female','count_register_gender_male','Year_dob','Month_dob','target']])

#####count_children

In [ ]:
vard[count_cols.filter(like='children').columns]

In [ ]:
Train[['count_children_present','count_register_all','count_children_attendance','count_children_precovid']]

In [ ]:
Train[['count_children_present','count_register_all','count_children_attendance','count_children_precovid','count_toilets_children']].describe()

In [ ]:
heatmap(Train[['count_children_present','count_register_all','count_children_attendance','count_children_precovid']])

In [ ]:
Train.groupby(['target_cat'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()


In [ ]:
Train.groupby(['child_grant'])[['count_children_present','count_children_attendance','count_children_precovid']]

In [ ]:
Train.groupby(['child_grant'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()


In [ ]:
Train.groupby(['age_bin'],sort=True)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()

In [ ]:
Train.groupby(['child_years_in_programme'],sort=True)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()

In [ ]:
aa = Train.groupby(['child_observe_attentive'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()
ab = Train.groupby(['child_observe_interested'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()
ac = Train.groupby(['child_observe_concentrated'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()
ad = Train.groupby(['child_observe_diligent'],sort=False)[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children',
                                      'Black-Coloured-Ratio','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()

ab = aa.join(ab,lsuffix='_aa',rsuffix='_ab')
ad = ac.join(ad,lsuffix='_ac',rsuffix='_ad')
at = ab.join(ad)
at['average_children_present'] = np.mean(at.filter(like='present'),axis=1).values
at['average_children_attendance'] = np.mean(at.filter(like='attendance'),axis=1).values
at['average_children_precovid'] = np.mean(at.filter(like='precovid'),axis=1).values
at['average_toilets_children'] = np.mean(at.filter(like='toilets'),axis=1).values
at['average_Black-Coloured-Ratio'] = np.mean(at.filter(like='Ratio'),axis=1).values
at['average_staff_salary_paid'] = np.mean(at.filter(like='salary'),axis=1).values
at['average_staff_skills'] = np.mean(at.filter(like='skills'),axis=1).values
at['average_staff_gender_female'] = np.mean(at.filter(like='gender'),axis=1).values

at = at.filter(like='average')
at

###language_child

In [ ]:
Train['language_child'].isnull().sum()/len(Train)

In [ ]:
barchart(Train['language_child'],Train['target'])

In [ ]:
violinplot(x=Train['language_child'],y=Train['target'],hue=Train['target_cat'],scale='count',length=20,width=10)

In [ ]:
Train['language_child'].value_counts()

In [ ]:
grouped = Train.groupby(['language_child','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['language_child']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.drop(['count'],axis=1,inplace=True)
grouped

In [ ]:
Train.groupby(['language_child'])[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children']].mean()

Children who speak English, siSwati and Tshivenda are the majority in How many children are present at the ECD Programme today? ("count_children_present") and How many children usually attend the ECD Programme, on average? ("count_children_attendance"), but in the How many children usually attended the ECD programme before the start of COVID on average" Tshivenda had the highest count, followed by the English and the Sesotho languages, the Amount of Children from Tshivenda and Sesotho dropped. Afrikaans, isiXhosa and Xitsonga have the majority in How many childrens toilets are there? why?

In [ ]:
Train.groupby(['language_child'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female','count_staff_skills','count_staff_salary_paid']].mean()

siSwati, Sesotho, Xitsonga have the highest number of registered african race, Afrikaans and English have the highest number of registered coloured race, English, Afrikaans and Xitsonga have the highest number of female staffs, Afrikaans, English and siSwati have the staff with the highest skills, English, Sesotho and Xitsongs pay the staff more

In [ ]:
grouped = Train.groupby(['child_grant','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_grant']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')
#Afrikaans and English are the majority who said No to the child_grant
#isiXhosa and isiZulu are the majority who said Yes to the child_grant

In [ ]:
grouped = Train.groupby(['age_bin','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['age_bin']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')
#Setswana, isiZulu, isiXhosa are the major youngest children in the ECD_Programme
#Setswana, isiZulu, isiXhosa are the major children in the age_bin 1 in the ECD_Programme
#Setswana, isiZulu, isiXhosa are the major children in the age_bin 2 in the ECD_Programme
#isiZulu, isiXhosa, Afrikaans are the major children in the age_bin 3 in the ECD_Programme
#Afrikaans, isiXhosa, isiZulu are the major children in the age_bin 3 in the ECD_Programme


In [ ]:
grouped = Train.groupby(['child_years_in_programme','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_years_in_programme']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')
#IsiZulu, Setswana, IsiXhosa are the major children in the 1st year of the programme
#Setswana, IsiZulu, IsiXhosa are the major children in the 2nd year of the programme
#IsiXhosa, IsiZulu, Setswana are the major children in the 3rd year of the programme

In [ ]:
grouped = Train.groupby(['child_gender','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_gender']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')
#IziXhosa, IziZulu and Setswana all the way

In [ ]:
grouped = Train.groupby(['child_observe_attentive','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_observe_attentive']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['child_observe_interested','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_observe_interested']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['child_observe_concentrated','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_observe_concentrated']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
ac = grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['child_observe_diligent','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['child_observe_diligent']).size()
grouped['norm_count'] = grouped['count'] / total_counts[grouped.index.get_level_values(0)].values
ad = grouped.filter(like='norm')

##ID

In [ ]:
Train['prov_best'].value_counts()

In [ ]:
grouped = Train.groupby(['prov_best','target_cat'])[['target_cat']].count().rename(columns={'target_cat':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')

    Provinces with higher proportions of children achieving the standard: Provinces such as Mpumalanga (0.585477), Western Cape (0.552212), and Gauteng (0.518057) have higher proportions of children achieving the standard compared to other provinces. This suggests that these provinces may have better educational outcomes for children, or better access to resources and opportunities that contribute to higher achievement levels.

    Provinces with higher proportions of children falling behind or at risk: Provinces such as Limpopo (0.317443 falling behind, 0.309859 at risk), Northern Cape (0.268139 falling behind, 0.364353 at risk), and Free State (0.257206 falling behind, 0.379157 at risk) have higher proportions of children falling behind or at risk compared to other provinces. This indicates that these provinces may have challenges in terms of educational outcomes or access to resources, which result in higher proportions of children facing difficulties in meeting the standard.

    Provinces with similar proportions across target categories: Provinces such as Eastern Cape (similar proportions across all target categories), KwaZulu-Natal (similar proportions across all target categories), North West (similar proportions across all target categories), and Unknown (similar proportions across all target categories) have relatively similar proportions of children across different target categories. This suggests that the educational outcomes for children in these provinces may be relatively balanced or similar across the three target categories.

    Variation in proportions across provinces: The data also reveals that there is variation in the proportions of children in different target categories across provinces, with some provinces having higher or lower proportions compared to others. This highlights the importance of considering provincial-level differences when analyzing educational outcomes and developing interventions or policies to address the needs of children in different regions.

In [ ]:
grouped = Train.groupby(['prov_best','language_child'])[['language_child']].count().rename(columns={'language_child':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')

    Language dominance: The data shows that English is the dominant language in most provinces, except for Eastern Cape where isiXhosa is the most commonly spoken language. This suggests that English is widely used and spoken across different provinces, which could be due to historical and cultural factors, as well as its status as a global language.

    Regional language variations: The data also reveals that certain provinces have a higher prevalence of specific regional languages. For example, Sesotho is prominent in Free State and Northern Cape, while Setswana is dominant in North West province. This suggests that there are regional language variations within South Africa, which may be influenced by factors such as geography, history, and cultural heritage.

    Language preservation: The data shows that some provinces have a higher proportion of indigenous languages, such as isiXhosa in Eastern Cape and isiZulu in KwaZulu-Natal, which indicates the efforts to preserve and maintain local languages. This highlights the importance of cultural preservation and recognition of linguistic diversity in South Africa.

    Language accessibility: The data also reveals that there are variations in language accessibility across provinces, with some languages being less commonly spoken or represented. For example, Afrikaans is less prevalent in some provinces compared to others, which could indicate differences in language accessibility and usage patterns based on historical, cultural, and socio-economic factors.

    Normative language use: The data provides insights into the normative language use in different provinces, with some languages being more commonly used in certain contexts or settings. For example, Afrikaans is more dominant in Western Cape, while Sepedi and Setswana are more prevalent in Mpumalanga and North West provinces, respectively. This suggests that language use is influenced by various factors such as socio-economic status, education, and cultural norms.

    Unknown language usage: The data also reveals that there are instances of "Unknown" language usage, particularly in the province of Gauteng. This could indicate challenges in accurately capturing and categorizing language data, and the need for further investigation and clarification to better understand language dynamics in the region.

In conclusion, the data provides valuable insights into language usage patterns in different provinces of South Africa, highlighting the dominance of English, regional language variations, efforts towards language preservation, variations in language accessibility, normative language use, and challenges in capturing accurate language data. These insights can be useful for policymakers, educators, and researchers in understanding language dynamics in South Africa and addressing issues related to language policy, cultural preservation, and linguistic diversity.

In [ ]:
grouped = Train.groupby(['prov_best','age_bin'])[['age_bin']].count().rename(columns={'age_bin':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='norm')

    Provinces with higher percentages of young children: Provinces like Free State, Limpopo, Mpumalanga, and Northern Cape have a higher percentage of young children in the 0-1 age group, which could indicate higher birth rates or larger populations of young families in these provinces. This may have implications for healthcare services, early childhood education, and other services targeted towards young children and their families.

    Consistent age distribution patterns: Provinces like Gauteng and KwaZulu-Natal have similar normalized counts across different age groups, indicating a relatively consistent age distribution pattern. This may suggest that these provinces have a stable population structure and may require similar policy interventions and resource allocations for different age groups.

    Variation in age distribution within provinces: Provinces like Free State and Northern Cape have more variation in normalized counts across different age groups, indicating differences in age demographics within the province. This may suggest the need for tailored policies and services to address the specific needs of different age groups within these provinces.

    Data gaps and missing information: The "Unknown" category with a significant percentage of people in the 0-1 age group may indicate data gaps or missing information in certain provinces. This highlights the need for further investigation and validation of data quality and reliability, and cautious interpretation of insights based on this data.

In [ ]:
grouped = Train.groupby(['prov_best','child_observe_attentive'])[['child_observe_attentive']].count().rename(columns={'child_observe_attentive':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
at = grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['prov_best','child_observe_concentrated'])[['child_observe_concentrated']].count().rename(columns={'child_observe_concentrated':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
ac = grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['prov_best','child_observe_interested'])[['child_observe_interested']].count().rename(columns={'child_observe_interested':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
ai = grouped.filter(like='norm')

In [ ]:
grouped = Train.groupby(['prov_best','child_observe_diligent'])[['child_observe_diligent']].count().rename(columns={'child_observe_diligent':'count'})
total_counts = Train.groupby(['prov_best']).size()
grouped['norm_count'] = grouped['count']/total_counts[grouped.index.get_level_values(0)].values
ad = grouped.filter(like='norm')

In [ ]:
at['child_observe_concentrated'] = ac['norm_count']
at['child_observe_interested'] = ai['norm_count']
at['child_observe_diligent'] = ad['norm_count']
at = at.rename(columns={'norm_count':'child_observe_att'})

In [ ]:
at['average_observe_levels'] = at.filter(like='observe').mean(axis=1)
at.filter(like='average')

    Regional Disparities: The data suggests that there may be regional disparities in the frequency of child observation levels in South Africa, with some provinces showing higher levels of child observation compared to others. This could be indicative of differences in child care practices, access to resources, and socio-economic conditions across different regions in the country.

    Policy Interventions: The data could be used to support the need for targeted policy interventions to improve child observation levels, particularly in provinces with lower average observation levels. For example, provinces with lower levels of child observation could be prioritized for interventions such as parenting programs, early childhood development initiatives, and caregiver support services to promote positive child observation behaviors.

    Further Research: The data could also highlight the need for further research to explore the underlying factors that contribute to the observed variations in child observation levels across different provinces in South Africa. Factors such as cultural norms, caregiver attitudes and beliefs, and contextual factors could be explored to better understand the reasons behind the observed patterns.


In [ ]:
Train.groupby(['prov_best'])[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children','count_staff_skills',]].mean()

In [ ]:
Train.groupby(['prov_best'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female','count_staff_salary_paid','target']].mean()

In [ ]:
vard[Train.filter(like='id_').columns[0:4]]

In [ ]:
vard[Train.filter(like='id_').columns[35:40]]

In [ ]:
Train[['id_mn_best','id_dc_best','id_enumerator','id_facility']]

In [ ]:
Train[['id_facility_n','id_ward_n','id_mn_n','id_dc_n','id_prov_n']]

In [ ]:
id_counts=Train.groupby(['prov_best'])[['id_mn_best','id_dc_best','id_enumerator','id_facility']].transform('count')
Train['id_mn_best_count'] = id_counts['id_mn_best']
Train['id_dc_best_count'] = id_counts['id_dc_best']
Train['id_enumerator_count'] = id_counts['id_enumerator']
Train['id_facility_count'] = id_counts['id_facility']

In [ ]:
Train.groupby(['prov_best'])[['id_facility_n','id_ward_n','id_mn_n','id_dc_n','id_prov_n']].mean()

##pri

In [ ]:
pri_cols = Train.filter(like='pri')
missing_cols = plot_missing_cols(pri_cols,threshold=0.6,plot=False)

In [ ]:
pri_cols.drop(missing_cols,axis=1,inplace=True)

In [ ]:
vard[pri_cols.columns[:10]]

In [ ]:
pri_cols.filter(like='time')

In [ ]:
Train['pri_calc_time_close'] = pd.to_datetime(Train['pri_calc_time_close'],format='%H:%M')
Train['pri_calc_time_open'] = pd.to_datetime(Train['pri_calc_time_open'],format='%H:%M')
Train['pri_calc_diff_hours'] = (Train['pri_calc_time_close'] - Train['pri_calc_time_open']).dt.components['hours']
Train['pri_calc_diff_minutes'] = (Train['pri_calc_time_close'] - Train['pri_calc_time_open']).dt.components['minutes']

In [ ]:
sns.catplot(data=Train,x='pri_time_open_hours',y='target',kind='box')

In [ ]:
sns.catplot(data=Train,x='pri_time_close_hours',y='target',kind='box',height=7)

In [ ]:
sns.catplot(data=Train,x='pri_calc_diff_hours',y='target',kind='box',height=7)

In [ ]:
Train[['pri_holidays']].value_counts()

In [ ]:
Train.groupby(['pri_holidays'])[['target']].mean()

In [ ]:
Train.groupby(['pri_holidays','target_cat'])[['target_cat']].count()

In [ ]:
grouped = Train.groupby(['pri_holidays','prov_best'])[['prov_best']].count()
total_counts = Train.groupby(['pri_holidays']).size()
grouped['count'] = grouped['prov_best']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='count')

In [ ]:
Train.groupby(['pri_holidays'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female','count_staff_salary_paid']].mean()

In [ ]:
Train.groupby(['pri_holidays'])[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children','count_staff_skills',]].mean()

In [ ]:
Train.groupby(['pri_holidays'])[['pri_time_open_hours','pri_time_close_hours','pri_calc_diff_hours']].mean()

In [ ]:
Train.groupby(['prov_best'])[['pri_time_open_hours','pri_time_close_hours','pri_calc_diff_hours']].mean()

In [ ]:
vard[pri_cols.columns[10:25]]

In [ ]:
afrikaans = Train.groupby(['pri_language_1'])[['target']].mean().rename(columns={'target':'Afrikaans_target'})

english = Train.groupby(['pri_language_2'])[['target']].mean().rename(columns={'target':'English_target'})

isiXhosa = Train.groupby(['pri_language_4'])[['target']].mean().rename(columns={'target':'IsiXhosa_target'})

isiZulu = Train.groupby(['pri_language_5'])[['target']].mean().rename(columns={'target':'IsiZulu_target'})

sepedi = Train.groupby(['pri_language_6'])[['target']].mean().rename(columns={'target':'Sepedi_target'})

setswana = Train.groupby(['pri_language_7'])[['target']].mean().rename(columns={'target':'Setswana_target'})

sesotho = Train.groupby(['pri_language_8'])[['target']].mean().rename(columns={'target':'Sesotho_target'})

tshivenda = Train.groupby(['pri_language_10'])[['target']].mean().rename(columns={'target':'Tshivenda_target'})

xitsonga = Train.groupby(['pri_language_11'])[['target']].mean().rename(columns={'target':'Xitsonga_target'})

pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)


In [ ]:
grouped = Train.groupby(['pri_language_1','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_1']).size()
grouped['Afrikaans_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
afrikaans = grouped.filter(like='count')


grouped = Train.groupby(['pri_language_2','child_observe_attentive',])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_2']).size()
grouped['English_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
english = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_4','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_4']).size()
grouped['isiXhosa_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
isiXhosa = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_5','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_5']).size()
grouped['isiZulu_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
isiZulu = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_6','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_6']).size()
grouped['Sepedi_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
sepedi = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_7','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_7']).size()
grouped['Setswana_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
setswana = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_8','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_8']).size()
grouped['Sesotho_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
sesotho = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_10','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_10']).size()
grouped['Tshivenda_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
tshivenda = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_11','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_language_11']).size()
grouped['Xitsonga_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
xitsonga = grouped.filter(like='count')

language_attentive = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)
language_attentive = language_attentive.reset_index()

no_row1=language_attentive.loc[0] + language_attentive.loc[2]
no_row2 = language_attentive.loc[1] + language_attentive.loc[3]
yes_row1=language_attentive.loc[4] + language_attentive.loc[6]
yes_row2 = language_attentive.loc[5] + language_attentive.loc[7]

language_attentive.loc[0] = no_row1
language_attentive.loc[1] = no_row2
language_attentive.loc[2] = yes_row1
language_attentive.loc[3] = yes_row2

language_attentive.drop([4,5,6,7],axis=0,inplace=True)
language_attentive[language_attentive.filter(like='_count').columns] = language_attentive.filter(like='_count')/2

In [ ]:
grouped = Train.groupby(['pri_language_1','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_1']).size()
grouped['Afrikaans_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
afrikaans = grouped.filter(like='count')


grouped = Train.groupby(['pri_language_2','child_observe_concentrated',])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_2']).size()
grouped['English_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
english = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_4','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_4']).size()
grouped['isiXhosa_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
isiXhosa = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_5','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_5']).size()
grouped['isiZulu_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
isiZulu = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_6','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_6']).size()
grouped['Sepedi_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
sepedi = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_7','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_7']).size()
grouped['Setswana_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
setswana = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_8','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_8']).size()
grouped['Sesotho_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
sesotho = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_10','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_10']).size()
grouped['Tshivenda_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
tshivenda = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_11','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_language_11']).size()
grouped['Xitsonga_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
xitsonga = grouped.filter(like='count')

language_concentrated = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)
language_concentrated = language_concentrated.reset_index()

no_row1=language_concentrated.loc[0] + language_concentrated.loc[2]
no_row2 = language_concentrated.loc[1] + language_concentrated.loc[3]
yes_row1=language_concentrated.loc[4] + language_concentrated.loc[6]
yes_row2 = language_concentrated.loc[5] + language_concentrated.loc[7]

language_concentrated.loc[0] = no_row1
language_concentrated.loc[1] = no_row2
language_concentrated.loc[2] = yes_row1
language_concentrated.loc[3] = yes_row2

language_concentrated.drop([4,5,6,7],axis=0,inplace=True)
language_concentrated[language_concentrated.filter(like='_count').columns] = language_concentrated.filter(like='_count')/2

In [ ]:
grouped = Train.groupby(['pri_language_1','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_1']).size()
grouped['Afrikaans_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
afrikaans = grouped.filter(like='count')


grouped = Train.groupby(['pri_language_2','child_observe_diligent',])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_2']).size()
grouped['English_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
english = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_4','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_4']).size()
grouped['isiXhosa_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
isiXhosa = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_5','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_5']).size()
grouped['isiZulu_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
isiZulu = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_6','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_6']).size()
grouped['Sepedi_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
sepedi = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_7','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_7']).size()
grouped['Setswana_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
setswana = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_8','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_8']).size()
grouped['Sesotho_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
sesotho = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_10','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_10']).size()
grouped['Tshivenda_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
tshivenda = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_11','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_language_11']).size()
grouped['Xitsonga_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
xitsonga = grouped.filter(like='count')

language_diligent = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)
language_diligent = language_diligent.reset_index()

no_row1=language_diligent.loc[0] + language_diligent.loc[2]
no_row2 = language_diligent.loc[1] + language_diligent.loc[3]
yes_row1=language_diligent.loc[4] + language_diligent.loc[6]
yes_row2 = language_diligent.loc[5] + language_diligent.loc[7]

language_diligent.loc[0] = no_row1
language_diligent.loc[1] = no_row2
language_diligent.loc[2] = yes_row1
language_diligent.loc[3] = yes_row2

language_diligent.drop([4,5,6,7],axis=0,inplace=True)
language_diligent[language_diligent.filter(like='_count').columns] = language_diligent.filter(like='_count')/2

In [ ]:
grouped = Train.groupby(['pri_language_1','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_1']).size()
grouped['Afrikaans_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
afrikaans = grouped.filter(like='count')


grouped = Train.groupby(['pri_language_2','child_observe_interested',])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_2']).size()
grouped['English_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
english = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_4','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_4']).size()
grouped['isiXhosa_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
isiXhosa = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_5','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_5']).size()
grouped['isiZulu_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
isiZulu = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_6','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_6']).size()
grouped['Sepedi_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
sepedi = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_7','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_7']).size()
grouped['Setswana_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
setswana = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_8','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_8']).size()
grouped['Sesotho_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
sesotho = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_10','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_10']).size()
grouped['Tshivenda_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
tshivenda = grouped.filter(like='count')

grouped = Train.groupby(['pri_language_11','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_language_11']).size()
grouped['Xitsonga_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
xitsonga = grouped.filter(like='count')

language_interested = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)
language_interested = language_interested.reset_index()

no_row1=language_interested.loc[0] + language_interested.loc[2]
no_row2 = language_interested.loc[1] + language_interested.loc[3]
yes_row1=language_interested.loc[4] + language_interested.loc[6]
yes_row2 = language_interested.loc[5] + language_interested.loc[7]

language_interested.loc[0] = no_row1
language_interested.loc[1] = no_row2
language_interested.loc[2] = yes_row1
language_interested.loc[3] = yes_row2

language_interested.drop([4,5,6,7],axis=0,inplace=True)
language_interested[language_interested.filter(like='_count').columns] = language_interested.filter(like='_count')/2

In [ ]:
average_language_observe = language_interested.copy()
average_language_observe[average_language_observe.filter(like='count').columns] = (language_interested.filter(like='count') +
                                                                                   language_attentive.filter(like='count') +
                                                                                   language_concentrated.filter(like='count') +
                                                                                   language_diligent.filter(like='count'))/5
average_language_observe
#Positive Observe levels tend to go up when the language for learning and teaching are Afrikaans, English, isiZulu, Sepedi, Tshivenda, Xitsonga
#Negative Observe levels tend to go up when the language for learning and teaching are isiXhosa, Setswana, Sesotho

In [ ]:
grouped = Train.groupby(['prov_best','pri_language_1'])[['pri_language_1']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Afrikaans_count'] = grouped['pri_language_1']/total_counts[grouped.index.get_level_values(0)].values
afrikaans = grouped.filter(like='count')


grouped = Train.groupby(['prov_best','pri_language_2'])[['pri_language_2']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['English_count'] = grouped['pri_language_2']/total_counts[grouped.index.get_level_values(0)].values
english = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_4',])[['pri_language_4']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['isiXhosa_count'] = grouped['pri_language_4']/total_counts[grouped.index.get_level_values(0)].values
isiXhosa = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_5',])[['pri_language_5']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['isiZulu_count'] = grouped['pri_language_5']/total_counts[grouped.index.get_level_values(0)].values
isiZulu = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_6'])[['pri_language_6',]].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Sepedi_count'] = grouped['pri_language_6']/total_counts[grouped.index.get_level_values(0)].values
sepedi = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_7'])[['pri_language_7']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Setswana_count'] = grouped['pri_language_7']/total_counts[grouped.index.get_level_values(0)].values
setswana = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_8'])[['pri_language_8']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Sesotho_count'] = grouped['pri_language_8']/total_counts[grouped.index.get_level_values(0)].values
sesotho = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_10'])[['pri_language_10']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Tshivenda_count'] = grouped['pri_language_10']/total_counts[grouped.index.get_level_values(0)].values
tshivenda = grouped.filter(like='count')

grouped = Train.groupby(['prov_best','pri_language_11'])[['pri_language_11']].count()
total_counts = Train.groupby(['prov_best']).size()
grouped['Xitsonga_count'] = grouped['pri_language_11']/total_counts[grouped.index.get_level_values(0)].values
xitsonga = grouped.filter(like='count')


In [ ]:
language_count = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga],axis=1)
language_count

In [ ]:
afrikaans = Train.groupby(['pri_language_1'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
afrikaans['language'] = 'Afrikaans'

english = Train.groupby(['pri_language_2'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
english['language'] = 'English'

isiXhosa = Train.groupby(['pri_language_4'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
isiXhosa['language'] = 'isiXhosa'

isiZulu = Train.groupby(['pri_language_5'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
isiZulu['language'] = 'isiZulu'

sepedi = Train.groupby(['pri_language_6'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
sepedi['language'] = 'Sepedi'

setswana = Train.groupby(['pri_language_7'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
setswana['language'] = 'Setswana'

sesotho = Train.groupby(['pri_language_8'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
sesotho['language'] = 'Sesotho'

tshivenda = Train.groupby(['pri_language_10'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
tshivenda['language'] = 'Tshivenda'

xitsonga = Train.groupby(['pri_language_11'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours']].mean()
xitsonga['language'] = 'Xitsonga'

In [ ]:
language = pd.concat([afrikaans,english,isiXhosa,isiZulu,sepedi,setswana,sesotho,tshivenda,xitsonga])
language = language.reset_index()
language.set_index(['language','index'])

In [ ]:
Train['pri_separate'].value_counts()

In [ ]:
grouped = Train.groupby(['pri_separate','target_cat'])[['target_cat']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['count'] = grouped['target_cat']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='count')

In [ ]:
grouped = Train.groupby(['pri_separate','child_observe_attentive'])[['child_observe_attentive']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['child_observe_attentive_count'] = grouped['child_observe_attentive']/total_counts[grouped.index.get_level_values(0)].values
att = grouped.filter(like='count')

grouped = Train.groupby(['pri_separate','child_observe_concentrated'])[['child_observe_concentrated']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['child_observe_concentrated_count'] = grouped['child_observe_concentrated']/total_counts[grouped.index.get_level_values(0)].values
ac = grouped.filter(like='count')

grouped = Train.groupby(['pri_separate','child_observe_interested'])[['child_observe_interested']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['child_observe_interested_count'] = grouped['child_observe_interested']/total_counts[grouped.index.get_level_values(0)].values
ai = grouped.filter(like='count')

grouped = Train.groupby(['pri_separate','child_observe_diligent'])[['child_observe_diligent']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['child_observe_diligent_count'] = grouped['child_observe_diligent']/total_counts[grouped.index.get_level_values(0)].values
ad = grouped.filter(like='count')

att = pd.concat([att,ac['child_observe_concentrated_count'],ai['child_observe_interested_count'],ad['child_observe_diligent_count']],axis=1)

In [ ]:
att.filter(like='count').mean(axis=1)

In [ ]:
grouped = Train.groupby(['pri_separate','prov_best'])[['prov_best']].count()
total_counts = Train.groupby(['pri_separate']).size()
grouped['count'] = grouped['prov_best']/total_counts[grouped.index.get_level_values(0)].values
grouped.filter(like='count')

In [ ]:
Train.groupby(['pri_separate'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female','count_staff_salary_paid']].mean()

In [ ]:
Train.groupby(['pri_separate'])[['count_children_present','count_children_attendance','count_children_precovid','count_toilets_children','count_staff_skills',]].mean()

In [ ]:
Train.groupby(['pri_separate'])[['pri_time_open_hours','pri_time_close_hours','pri_calc_diff_hours']].mean()

In [ ]:
vard[pri_cols.columns[25:35]]

In [ ]:
Train.groupby(['pri_transport'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train.groupby(['pri_meal'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train.groupby(['pri_bank'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train.groupby(['pri_land'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train.groupby(['pri_facilities'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train[['pri_fees','pri_fees_amount','pri_fees_free']]

In [ ]:
print(Train['pri_fees'].value_counts()/len(Train['pri_fees']))
print(Train['pri_fees_free'].value_counts()/len(Train['pri_fees_free']))

In [ ]:
Train.groupby(['pri_fees'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train.groupby(['pri_fees_free'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
Train['pri_aftercare'].value_counts()

In [ ]:
Train.groupby(['pri_aftercare'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

In [ ]:
vard[pri_cols.columns[35:45]]

In [ ]:
Train[['pri_meal_1','pri_meal_3','pri_meal_4','pri_meal_2','pri_meals']]

In [ ]:
Train['pri_meals'].value_counts()

In [ ]:
Train.groupby(['pri_meals'])[['count_register_race_african','count_register_race_coloured','count_staff_gender_female',
                                   'count_staff_salary_paid','count_children_present','count_children_attendance','count_children_precovid',
                                   'count_toilets_children','count_staff_skills','pri_time_open_hours','pri_time_close_hours','pri_fees_amount']].mean()

#Feature Engineering

In [ ]:
def logic_mapping(att, conc, dilig, interest):
  logic_map = {'Sometimes':False,'Often':True,'Almost always':True,'Almost never':False}
  result_and = (logic_map[att]) and (logic_map[conc]) and (logic_map[dilig]) and (logic_map[interest])
  result_or = (logic_map[att]) or (logic_map[conc]) or (logic_map[dilig]) or (logic_map[interest])
  result_nand = (not(logic_map[att])) and (not(logic_map[conc])) and (not(logic_map[dilig])) and (not(logic_map[interest]))
  result_nor = (not(logic_map[att])) and (not(logic_map[conc])) and (not(logic_map[dilig])) and (not(logic_map[interest]))
  logic_remap = {True:0,False:1}
  return logic_remap[result_and], logic_remap[result_or], logic_remap[result_nand], logic_remap[result_nor]

In [ ]:
attention_mapping = {'Sometimes': 2, 'Often': 3, 'Almost always': 4, 'Almost never': 1}
programme_year_mapping = {'1st year in the programme':1, '2nd year in programme': 2, '3rd year in programme':3, 'Do Not Know':4}
Logic_ops = data[['child_observe_attentive','child_observe_concentrated','child_observe_diligent','child_observe_interested']].apply(
    lambda row: logic_mapping(row[0],row[1],row[2],row[3]), axis=1)
data['AND_Observe'] = Logic_ops.apply(lambda row: row[0])
data['OR_Observe'] = Logic_ops.apply(lambda row: row[1])
data['NAND_Observe'] = Logic_ops.apply(lambda row: row[2])
data['NOR_Observe'] = Logic_ops.apply(lambda row: row[3])
data['child_years_in_programme'] = data['child_years_in_programme'].map(programme_year_mapping)
data['age_bin'] = pd.qcut(data['child_age'],5,labels=False)
data['child_observe_attentive'] = data['child_observe_attentive'].map(attention_mapping)
data['child_observe_concentrated'] = data['child_observe_concentrated'].map(attention_mapping)
data['child_observe_diligent'] = data['child_observe_diligent'].map(attention_mapping)
data['child_observe_interested'] = data['child_observe_interested'].map(attention_mapping)
data['child_observe_totals'] = data['child_observe_diligent'] * 4 + data['child_observe_attentive'] * 3 + data['child_observe_concentrated'] * 2 + data['child_observe_interested'] * 1
data['child_observe_totals*'] = (data['child_observe_diligent'] * 4) * (data['child_observe_attentive'] * 3) * (data['child_observe_concentrated'] * 2) * (data['child_observe_interested'] * 1)
data['weighted_child_age+attention'] = data['child_observe_diligent'] + data['age_bin'] + 1 + data['child_observe_attentive']
data['weighted_child_age*attention'] = data['child_observe_diligent'] * (data['age_bin']+1) * data['child_observe_attentive']
data['weighted_child_age+prog_year+attention'] = data['child_years_in_programme'] + data['child_observe_diligent'] + data['age_bin'] + 1 + data['child_observe_attentive']
data['weighted_child_age*prog_year*attention'] = data['child_years_in_programme'] * data['child_observe_diligent'] * (data['age_bin']+1) * data['child_observe_attentive']

In [ ]:
data['Year_dob'] = data['child_dob'].str.split('-',expand=True)[0].astype(float)
data['Month_dob'] = data['child_dob'].str.split('-',expand=True)[1].astype(float)
data['Day_dob'] = data['child_dob'].str.split('-',expand=True)[2].astype(float)
data['child_gender'] = data['child_gender'].map({'Female':1,'Male':2})

In [ ]:
data['Count_observe_totals_level_per_age_group'] = data.groupby(['age_bin','child_observe_totals'])[['child_observe_totals']].transform('count')

In [ ]:
data['Mean_observe_totals_level_per_age_group'] = data.groupby(['age_bin','child_observe_totals'])[['child_observe_totals']].transform('mean')

In [ ]:
data['sum_observe_totals_level_per_age_group'] = data.groupby(['age_bin','child_observe_totals'])[['child_observe_totals']].transform('sum')

In [ ]:
data['Mean_weighted_attentive_level_per_gender'] = data.groupby(['child_gender'])[['weighted_child_age+prog_year+attention']].transform('mean')

In [ ]:
data['Mean_weighted_attentive*_level_per_gender'] = data.groupby(['child_gender'])[['weighted_child_age*prog_year*attention']].transform('mean')

In [ ]:
new_Train = data[data['Train']=='Train']
new_Test = data[data['Train']=='Test']

#Baseline Score

In [ ]:
def Baseline_Pred(model, X, y, test, scaler=None, verbose=False, predictions=True):
  fold = KFold(n_splits=5)
  test_preds = []
  oof_f1 = []
  for train_index, test_index in fold.split(X, y):
    X_train, X_test = X.iloc[train_index],X.iloc[test_index]
    y_train, y_test = y.iloc[train_index],y.iloc[test_index]
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=verbose)
    preds=model.predict(X_test)
    if predictions:
      test_pred = model.predict(test[X.columns])
      test_preds.append(test_pred)
    print(f"RMSE {mean_squared_error(y_test,preds,squared=False)}")
    oof_f1.append(mean_squared_error(y_test,preds,squared=False))

  print(f"Average_RMSE {sum(oof_f1)/5}")
  if predictions:
    return test_preds, model
  return model

###Numerical Columns

In [ ]:
# num_cols = list(set(Test.select_dtypes('number')).intersection(Train.select_dtypes('number')))
num_cols = new_Train.select_dtypes('number').columns
num_train = new_Train[num_cols]
#num_test = Test[num_cols]

In [ ]:
missing_train_cols= plot_missing_cols(num_train, threshold=0.8, plot=False)
#missing_test_cols = plot_missing_cols(num_test, threshold=0.8, plot=False)
num_Train = num_train.drop(missing_train_cols,axis=1)
#num_Test = num_test.drop(missing_test_cols,axis=1)

In [ ]:
model_cbc = CatBoostRegressor(eval_metric='RMSE')
model_lgb = LGBMRegressor()

In [ ]:
num_Train.drop(['target','child_age','child_height','child_observe_total'],axis=1,inplace=True)

In [ ]:
cbc_model = Baseline_Pred(model_cbc, num_Train, Train['target'], Test, verbose=False, predictions=False)

In [ ]:
features_cbc = pd.DataFrame({'feature': list(num_Train.columns), 'importance': cbc_model.feature_importances_}).sort_values('importance', ascending = False)
features_cbc.head(30)


###Categorical Columns

In [ ]:

cat_cols = Train.select_dtypes(include=['object']).columns


In [ ]:
le = LabelEncoder()

In [ ]:
test_cat_cols = Test.select_dtypes('object').columns

In [ ]:
cat_train = Train[cat_cols]
cat_test = Test[test_cat_cols]
missing_cols_train = plot_missing_cols(cat_train,threshold=0.8,plot=False)
missing_cols_test = plot_missing_cols(cat_test, threshold=0.8,plot=False)

In [ ]:
cat_Train = cat_train.drop(missing_cols_train,axis=1)
cat_Test = cat_test.drop(missing_cols_test,axis=1)

In [ ]:
for col in cat_Train.columns:
  cat_Train[col] = le.fit_transform(cat_Train[col])

In [ ]:
for col in cat_Test.columns:
  cat_Test[col] = le.fit_transform(cat_Test[col])

In [ ]:
cat_Train.drop(['child_id'],axis=1,inplace=True)
cat_Test.drop(['child_id'],axis=1,inplace=True)

In [ ]:
model_cbc = CatBoostRegressor(eval_metric='RMSE')

In [ ]:
cbc_preds, cbc_model = Baseline_Pred(model_cbc, cat_Train, Train['target'], cat_Test)

##Submission

In [ ]:
explainer = shap.Explainer(cbc_model)

In [ ]:
# Compute the SHAP values for your test dataset
shap_values = explainer(num_Test)

In [ ]:
xd = num_Test.copy()
xd[num_Test.columns] = shap_values.values

In [ ]:
top_cols = xd.apply(lambda x: x.nlargest(15).index.tolist(), axis=1)

In [ ]:
feature_cols = Sub.filter(like='feature').columns

In [ ]:
xdc = pd.DataFrame(columns=feature_cols)
for i, cols in enumerate(top_cols):
  xdc.loc[i] = cols

In [ ]:
Sub['target'] = sum(cbc_preds)/5

In [ ]:
Sub[feature_cols] = xdc[feature_cols]

In [ ]:
Sub.to_csv('/content/Baseline_Only_Numerical_Columns_above_0.8threshold_CatBoost_Sub.csv',index=False)